<a href="https://colab.research.google.com/github/rahiakela/fluent-python-book-practice/blob/6-design-patterns-with-first-class-functions/1_classic_strategy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Design patterns with first-class functions

Although design patterns are language-independent, that does not mean every pattern applies to every language. In his 1996, Peter Norvig states that 16 out of the 23 patterns in the original Design Patterns book by Gamma et.al. become either “invisible or simpler” in a dynamic language.

In particular, in the context of languages with first-class functions, Norvig suggests rethinking the Strategy, Command, Template Method and Visitor patterns. 

The general idea is: you can replace instances of some participant class in these patterns with simple functions, reducing a lot of boilerplate code.

So we will refactor Strategy using function objects, and discuss a similar approach to simplifying the Command pattern.

## Case study: refactoring Strategy

Strategy is a good example of a design pattern that can be simpler in Python if you leverage functions as first class objects. In the following section we describe and implement
Strategy using the “classic” structure described in the Design patterns book.

<img src='https://github.com/rahiakela/img-repo/blob/master/strategy-design-pattern.png?raw=1' width='800'/>

A clear example of Strategy applied in the e-commerce domain is computing discounts to orders according to the attributes of the customer or inspection of the ordered items.

Consider an online store with these discount rules:

* customers with 1000 or more fidelity points get a global 5% discount per order;
* a 10% discount is applied to each line item with 20 or more units in the same order;
* orders with at least 10 distinct items get a 7% global discount.

For brevity, let us assume that only one discount may be applied to an order.

Strategy pattern participants are:
* **Context**
    * Provides a service by delegating some computation to interchangeable components that implement alternative algorithms. In the e-commerce example, the context is an Order, which is configured to apply a promotional discount according to one of several algorithms.

* **Strategy**
    * The interface common to the components that implement the different algorithms. In our example, this role is played by an abstract class called Promotion.

* **Concrete Strategy**
    * One of the concrete subclasses of Strategy. FidelityPromo, BulkPromo and Large OrderPromo are the three concrete strategies implemented.

The concrete strategy is chosen by the client of the context class. In our example, before instantiating an order, the system would somehow select a promotional discount strategy and pass it to the Order constructor. The selection of the strategy is outside of the scope of the pattern.


In [0]:
from abc import ABC, abstractmethod
from collections import namedtuple

Customer = namedtuple('Customer', 'name fidelity')

class LineItem:
  def __init__(self, product, quantity, price):
    self.product = product
    self.quantity = quantity
    self.price = price

  def total(self):
    return self.price * self.quantity

# the Context
